In [60]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import os
import pandas as pd
from pymystem3 import Mystem
import re
from sklearn.model_selection import train_test_split
import string
from tqdm import tqdm

# Download data

In [ ]:
!wget https://github.com/ilyasssklimov/misis_nlp_project/raw/main/data/raw_data.rar

--2024-05-11 18:02:08--  https://github.com/ilyasssklimov/misis_nlp_project/raw/main/data/raw_data.rar
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ilyasssklimov/misis_nlp_project/main/data/raw_data.rar [following]
--2024-05-11 18:02:08--  https://raw.githubusercontent.com/ilyasssklimov/misis_nlp_project/main/data/raw_data.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33053615 (32M) [application/octet-stream]
Saving to: ‘raw_data.rar’

raw_data.rar        100%[===================>]  31.52M   120MB/s    in 0.3s    

2024-05-11 18:02:09 (120 MB/s) - ‘raw_data.rar’ saved [33053615/33053615

In [ ]:
!unrar e raw_data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from raw_data.rar

Extracting  articles.csv                                                  12% 25% 38% 50% 63% 76% 88%  OK 
Extracting  news.csv                                                     100%  OK 
All OK


In [ ]:
articles_df = pd.read_csv('articles.csv')
news_df = pd.read_csv('news.csv')

In [ ]:
articles_df.head()

,Unnamed: 0,name,publication_date,text,link,author,tags,topic
0,0,Python — Дескрипторы(Descriptors),2024-05-02 13:17:00,"Если обратиться к документации, то дескриптор ...",https://habr.com/ru/articles/811777/,KarmanovichDev,['Python'],develop
1,1,Как начать писать на Java в VSCode,2024-05-02 13:08:00,Давайте признаемся: подавляющее большинство пи...,https://habr.com/ru/companies/cdek_blog/articl...,sendelust,"['Блог компании CDEK', 'Java', 'Текстовые реда...",develop
2,2,Security Week 2418: приватность автотелеметрии,2024-05-02 13:06:00,На прошлой неделе издание New York Times показ...,https://habr.com/ru/companies/kaspersky/articl...,Kaspersky_Lab,"['Блог компании «Лаборатория Касперского»', 'И...",develop
3,3,Шаг за шагом: разработка 3D-игры в Godot 4.2 д...,2024-05-02 13:00:00,"И снова привет, Хабр! В прошлой статье я расск...",https://habr.com/ru/companies/selectel/article...,techno_mot,"['Блог компании Selectel', 'DIY или Сделай сам...",develop
4,4,Как организовать информационную безопасность в...,2024-05-02 12:50:00,"Привет! Меня зовут Андрей, и я руковожу команд...",https://habr.com/ru/companies/avito/articles/8...,toogle,"['Блог компании AvitoTech', 'Информационная бе...",develop


In [ ]:
articles_df.drop('Unnamed: 0', axis=1, inplace=True)
news_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
articles_df.head(1)

,name,publication_date,text,link,author,tags,topic
0,Python — Дескрипторы(Descriptors),2024-05-02 13:17:00,"Если обратиться к документации, то дескриптор ...",https://habr.com/ru/articles/811777/,KarmanovichDev,['Python'],develop


In [ ]:
news_df.head(1)

,name,publication_date,text,link,author,tags,topic
0,В Google Play появился значок для маркировки п...,2024-05-03 11:47:00,Google начала маркировать правительственные мо...,https://habr.com/ru/news/812007/,daniilshat,"['Разработка под Android', 'Разработка мобильн...",develop


In [ ]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6774 entries, 0 to 6773
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              6774 non-null   object
 1   publication_date  6774 non-null   object
 2   text              6774 non-null   object
 3   link              6774 non-null   object
 4   author            6754 non-null   object
 5   tags              6774 non-null   object
 6   topic             6774 non-null   object
dtypes: object(7)
memory usage: 370.6+ KB


In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6357 entries, 0 to 6356
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              6357 non-null   object
 1   publication_date  6357 non-null   object
 2   text              6357 non-null   object
 3   link              6357 non-null   object
 4   author            6357 non-null   object
 5   tags              6357 non-null   object
 6   topic             6357 non-null   object
dtypes: object(7)
memory usage: 347.8+ KB


In [ ]:
data_df = pd.concat([articles_df, news_df]).drop_duplicates(
  subset=['link', 'topic']
)
data_df.head()

,name,publication_date,text,link,author,tags,topic
0,Python — Дескрипторы(Descriptors),2024-05-02 13:17:00,"Если обратиться к документации, то дескриптор ...",https://habr.com/ru/articles/811777/,KarmanovichDev,['Python'],develop
1,Как начать писать на Java в VSCode,2024-05-02 13:08:00,Давайте признаемся: подавляющее большинство пи...,https://habr.com/ru/companies/cdek_blog/articl...,sendelust,"['Блог компании CDEK', 'Java', 'Текстовые реда...",develop
2,Security Week 2418: приватность автотелеметрии,2024-05-02 13:06:00,На прошлой неделе издание New York Times показ...,https://habr.com/ru/companies/kaspersky/articl...,Kaspersky_Lab,"['Блог компании «Лаборатория Касперского»', 'И...",develop
3,Шаг за шагом: разработка 3D-игры в Godot 4.2 д...,2024-05-02 13:00:00,"И снова привет, Хабр! В прошлой статье я расск...",https://habr.com/ru/companies/selectel/article...,techno_mot,"['Блог компании Selectel', 'DIY или Сделай сам...",develop
4,Как организовать информационную безопасность в...,2024-05-02 12:50:00,"Привет! Меня зовут Андрей, и я руковожу команд...",https://habr.com/ru/companies/avito/articles/8...,toogle,"['Блог компании AvitoTech', 'Информационная бе...",develop


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12999 entries, 0 to 6349
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              12999 non-null  object
 1   publication_date  12999 non-null  object
 2   text              12999 non-null  object
 3   link              12999 non-null  object
 4   author            12979 non-null  object
 5   tags              12999 non-null  object
 6   topic             12999 non-null  object
dtypes: object(7)
memory usage: 1.3+ MB


# Processing data

In [ ]:
data_df['topic'].value_counts()

topic
develop       2379
popsci        2324
management    2197
admin         2068
marketing     2024
design        2007
Name: count, dtype: int64

In [ ]:
data_df.groupby('link').size().reset_index()[0].value_counts()

0
1    6617
2    2201
3     565
4      70
5       1
Name: count, dtype: int64

In [ ]:
topics = ['admin', 'design', 'develop', 'management', 'marketing', 'popsci']
labels = ['name', 'link', 'text']

topics_df = pd.get_dummies(data_df['topic'])
data_df = pd.concat(
  [data_df[labels], topics_df], axis=1
).groupby('link').agg({
  'name': 'max',
  'text': 'max',
  **{topic: 'sum' for topic in topics}
}).reset_index()

In [ ]:
data_df.head()

,link,name,text,admin,design,develop,management,marketing,popsci
0,https://habr.com/ru/articles/702300/,"Программирование и ИТ во мгле, но это не точно",(Статья была на редактировании. исправил ошибк...,0,0,0,0,0,1
1,https://habr.com/ru/articles/721426/,Территория Большого Взрыва. Как устроена Вселе...,Немного обо мне и о темеЭта статья — первая из...,0,0,0,0,0,1
2,https://habr.com/ru/articles/722566/,Изометрическое Черчение в Inkscape,В разработке электроники часто надо строить пр...,0,1,0,0,0,0
3,https://habr.com/ru/articles/723298/,Территория Большого Взрыва. От трёх китов к из...,"Эта статья — вторая в серии, основанной на моё...",0,0,0,0,0,1
4,https://habr.com/ru/articles/736230/,Дизайн уровней в Dishonored,Dishonored меня всегда впечатляла не только св...,0,1,0,0,0,0


In [ ]:
data_df.shape

(9454, 9)

In [ ]:
data_df.groupby('link').size().reset_index()[0].value_counts()

0
1    9454
Name: count, dtype: int64

In [ ]:
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")

mystem = Mystem()


def preprocess_text(text: str) -> str:
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(fr'[{string.punctuation}]', ' ', text)

  tokens = re.findall(r'\w+', text)
  clear_text = ' '.join([token.lower() for token in tokens
                         if token not in russian_stopwords and token != ' '])

  lemmatize_tokens = mystem.lemmatize(clear_text)
  return re.sub(r'\s+', ' ', ' '.join(lemmatize_tokens).strip())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
data_df['text'].iloc[0]

'(Статья была на редактировании. исправил ошибки и убрал часть мемов, так как особо не несли нагрузку.)У меня не получилось объяснить этим вашим ГПТ, какую статью я хочу нагенерировать. Пришлось писать самому и придумывать мемы. Поэтому в статье, будет гиперболизация проблем в айти и программировании в частности, передергивания смыслов, набрасывание на вентилятор, утверждения без пруфов и шутки про гладиолус ибо я олд.Приветствую, Хабравчане!Немного о себе. Я скромный богобоязненный разработчик на С++/С#. В основном моя работа связана с бэкендом, оптимизацией SQL запросов, разработка архитектур баз данных и сервисов для обеспечения высоких нагрузок. В разработке около 10-ти лет. За это время я видел такое, что вам, людям не программистам, и не снилось. Атакующие сервер пользователи, пылающие сервера за стенкой; Указатели Си и ошибки с ними связанные, разрезающие человека на пополам. Все эти мгновения затеряются во времени, как... байты на 8 тб жестком диске... Пришло время рассказать..

In [ ]:
BATCH_SIZE = 100
SPLIT_WORD = 'endsentence'

texts = data_df['text'].tolist()
text_batches = [texts[i:(i + BATCH_SIZE)]
                for i in range(0, len(texts), BATCH_SIZE)]

preprocessed_texts = []
for text_batch in tqdm(text_batches):
  texts = ''.join([f'{text} {SPLIT_WORD} ' for text in text_batch])
  preprocessed_text = preprocess_text(texts).split(SPLIT_WORD)
  preprocessed_texts.extend([text.strip() for text in preprocessed_text
                             if text.strip()])

data_df['preprocessed_text'] = preprocessed_texts

100%|██████████| 95/95 [08:10<00:00,  5.16s/it]


In [ ]:
data_df['preprocessed_text'].iloc[0]

'статья редактирование исправлять ошибка убирать часть мем особо нести нагрузка у получаться объяснять этот ваш гпт какой статья хотеть нагенерировать приходиться писать сам придумывать мема поэтому статья гиперболизация проблема айти программирование частность передергивание смысл набрасывание вентилятор утверждение пруфов шутка гладиолус ибо олд приветствовать хабравчанин немного я скромный богобоязненный разработчик с с в основной работа связывать бэкенд оптимизация sql запрос разработка архитектура база данные сервис обеспечение высокий нагрузка в разработка около ти год за этот время видеть такой человек программист сниться атаковать сервер пользователь пылать сервер стенка указатель си ошибка они связывать разрезать человек пополам весь мгновение затериваться время байт тб жесткий диск приходить время рассказывать предлагать погружаться чарующий мир рукожопий всратость вы готовый погружаться пример обучение программированиив наш просвещенный век гигабитный подключение каждый дом 

In [ ]:
data_df.head(1)

,link,name,text,admin,design,develop,management,marketing,popsci,preprocessed_text
0,https://habr.com/ru/articles/702300/,"Программирование и ИТ во мгле, но это не точно",(Статья была на редактировании. исправил ошибк...,0,0,0,0,0,1,статья редактирование исправлять ошибка убират...


In [ ]:
data_df.to_csv('preprocessed_data.csv', index=False)

# Download preprocessed data

In [2]:
!wget https://github.com/ilyasssklimov/misis_nlp_project/raw/main/data/preprocessed_data.rar

--2024-05-21 16:29:00--  https://github.com/ilyasssklimov/misis_nlp_project/raw/main/data/preprocessed_data.rar
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ilyasssklimov/misis_nlp_project/main/data/preprocessed_data.rar [following]
--2024-05-21 16:29:00--  https://raw.githubusercontent.com/ilyasssklimov/misis_nlp_project/main/data/preprocessed_data.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44720633 (43M) [application/octet-stream]
Saving to: ‘preprocessed_data.rar’

preprocessed_data.r 100%[===================>]  42.65M  98.8MB/s    in 0.4s    

2024-05-21 16:29:01 (98.8 MB/s) - ‘p

In [3]:
!unrar e preprocessed_data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from preprocessed_data.rar

Extracting  preprocessed_data.csv                                          9% 18% 28% 37% 46% 56% 65% 75% 84% 93%100%  OK 
All OK


In [4]:
preprocessed_data_df = pd.read_csv('preprocessed_data.csv')
preprocessed_data_df.head()

,link,name,text,admin,design,develop,management,marketing,popsci,preprocessed_text
0,https://habr.com/ru/articles/702300/,"Программирование и ИТ во мгле, но это не точно",(Статья была на редактировании. исправил ошибк...,0,0,0,0,0,1,статья редактирование исправлять ошибка убират...
1,https://habr.com/ru/articles/721426/,Территория Большого Взрыва. Как устроена Вселе...,Немного обо мне и о темеЭта статья — первая из...,0,0,0,0,0,1,немного обо темеэт статья первый три основыват...
2,https://habr.com/ru/articles/722566/,Изометрическое Черчение в Inkscape,В разработке электроники часто надо строить пр...,0,1,0,0,0,0,в разработка электроника часто строить прототи...
3,https://habr.com/ru/articles/723298/,Территория Большого Взрыва. От трёх китов к из...,"Эта статья — вторая в серии, основанной на моё...",0,0,0,0,0,1,этот статья второй серия основывать мой курс т...
4,https://habr.com/ru/articles/736230/,Дизайн уровней в Dishonored,Dishonored меня всегда впечатляла не только св...,0,1,0,0,0,0,dishonored впечатлять свой игровой процесс пла...


In [5]:
preprocessed_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9454 entries, 0 to 9453
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   link               9454 non-null   object
 1   name               9454 non-null   object
 2   text               9454 non-null   object
 3   admin              9454 non-null   int64 
 4   design             9454 non-null   int64 
 5   develop            9454 non-null   int64 
 6   management         9454 non-null   int64 
 7   marketing          9454 non-null   int64 
 8   popsci             9454 non-null   int64 
 9   preprocessed_text  9454 non-null   object
dtypes: int64(6), object(4)
memory usage: 738.7+ KB


In [6]:
from sklearn.metrics import (
  accuracy_score, recall_score, precision_score, f1_score
)

def count_metrics(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred, average='micro')
  precision = precision_score(y_test, y_pred, average='micro')
  f1 = f1_score(y_test, y_pred, average='micro')

  return {
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1-micro': f1
  }

# CountVectorizer

In [24]:
topics = ['admin', 'design', 'develop', 'management', 'marketing', 'popsci']

X = preprocessed_data_df['preprocessed_text']
y = np.asarray(preprocessed_data_df[topics])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7563,)
(1891,)
(7563, 6)
(1891, 6)


## LogisticRegression

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


logreg = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(LogisticRegression())),
])

In [27]:
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=LogisticRegression()))])

In [28]:
y_pred = logreg.predict(X_test)

In [29]:
count_metrics(y_test, y_pred)

{'accuracy': 0.37123215230037016,
 'precision': 0.8181102362204724,
 'recall': 0.4017788089713844,
 'f1-micro': 0.5389004149377594}

## Gradient Boosting

In [30]:
from xgboost import XGBClassifier

In [31]:
xgb = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss')))
])

In [32]:
xgb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               device=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric='logloss',
                                                               feature_types=None,
                                                               gamma=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=None,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...)))])

In [33]:
y_pred = xgb.predict(X_test)

In [34]:
count_metrics(y_test, y_pred)

{'accuracy': 0.370174510840825,
 'precision': 0.7405572755417956,
 'recall': 0.46249033255993816,
 'f1-micro': 0.569388240895025}

# Word2Vec

In [35]:
from sklearn.preprocessing import FunctionTransformer
from gensim.models import Word2Vec


sentences = [text.split() for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100,
                     window=5, min_count=1, workers=4)
w2v_model

In [36]:
def text_to_w2v(text, vector_size=100):
  words = text.split()
  word_vectors = [w2v_model.wv[word] for word in words
                  if word in w2v_model.wv]
  if len(word_vectors) == 0:
      return np.zeros(vector_size)
  return np.mean(word_vectors, axis=0)

def w2v_transformer(texts):
  return np.array([text_to_w2v(text) for text in texts])

In [37]:
logreg = Pipeline([
  ('w2v', FunctionTransformer(w2v_transformer, validate=False)),
  ('clf', MultiOutputClassifier(LogisticRegression(max_iter=1000)))
])

In [38]:
logreg.fit(X_train, y_train)

Pipeline(steps=[('w2v',
                 FunctionTransformer(func=<function w2v_transformer at 0x7d478eebf760>)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000)))])

In [39]:
y_pred = logreg.predict(X_test)

In [40]:
count_metrics(y_test, y_pred)

{'accuracy': 0.3352723426758329,
 'precision': 0.6973599484867997,
 'recall': 0.41879350348027844,
 'f1-micro': 0.523314810340662}

In [48]:
xgb = Pipeline([
  ('w2v', FunctionTransformer(w2v_transformer, validate=False)),
  ('clf', MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss')))
])

In [49]:
xgb.fit(X_train, y_train)

Pipeline(steps=[('w2v',
                 FunctionTransformer(func=<function w2v_transformer at 0x7d478eebf760>)),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               device=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric='logloss',
                                                               feature_...
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=None,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None, ...)))])

In [50]:
y_pred = xgb.predict(X_test)

In [51]:
count_metrics(y_test, y_pred)

{'accuracy': 0.40190375462718136,
 'precision': 0.6958845537145911,
 'recall': 0.5034802784222738,
 'f1-micro': 0.5842494951761275}

# BERT

In [61]:
from numpy import asarray
import os
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from typing import Dict
from tqdm.notebook import tqdm

In [62]:
class ModelForClassification(torch.nn.Module):
    def __init__(self, model_path: str, config: Dict):
        super(ModelForClassification, self).__init__()
        self.model_name = model_path
        self.config = config
        self.n_classes = config['num_classes']
        self.dropout_rate = config['dropout_rate']
        self.bert = AutoModel.from_pretrained(self.model_name)
        self.pre_classifier = torch.nn.Linear(312, 768)
        self.dropout = torch.nn.Dropout(self.dropout_rate)
        self.classifier = torch.nn.Linear(768, self.n_classes)

    def forward(self, input_ids, attention_mask,):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        hidden_state = output[0]
        hidden_state = hidden_state[:, 0]
        hidden_state = self.pre_classifier(hidden_state)
        hidden_state = torch.nn.ReLU()(hidden_state)
        hidden_state = self.dropout(hidden_state)
        output = self.classifier(hidden_state)
        return output

In [63]:
class Trainer:
    def __init__(self, config: Dict):
        self.config = config
        self.n_epochs = config['n_epochs']
        self.optimizer = None
        self.opt_fn = lambda model: Adam(model.parameters(), config['lr'])
        self.model = None
        self.history = None
        self.loss_fn = CrossEntropyLoss()
        self.device = config['device']
        self.verbose = config.get('verbose', True)

    def fit(self, model, train_dataloader):
        self.model = model.to(self.device)
        self.optimizer = self.opt_fn(model)
        self.history = {
            'train_loss': []
        }

        for epoch in range(self.n_epochs):
            print(f"Epoch {epoch + 1}/{self.n_epochs}")
            train_info = self.train_epoch(train_dataloader)
            self.history['train_loss'].extend(train_info['loss'])

        return self.model.eval()

    def train_epoch(self, train_dataloader):
        self.model.train()
        losses = []
        if self.verbose:
            train_dataloader = tqdm(train_dataloader)
        for batch in train_dataloader:
            ids = batch['ids'].to(self.device, dtype=torch.long)
            mask = batch['mask'].to(self.device, dtype=torch.long)
            targets = batch['targets'].to(self.device, dtype=torch.long)

            outputs = self.model(ids, mask)
            loss = self.loss_fn(outputs, targets)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_val = loss.item()
            if self.verbose:
                train_dataloader.set_description(f"Loss={loss_val:.3}")
            losses.append(loss_val)
        return {'loss': losses}

    def predict(self, test_dataloader):
        if not self.model:
            raise RuntimeError("You should train the model first")
        self.model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_dataloader:
                ids = batch['ids'].to(self.device, dtype=torch.long)
                mask = batch['mask'].to(self.device, dtype=torch.long)
                outputs = self.model(ids, mask)
                predictions.extend(outputs.argmax(1).tolist())
        return asarray(predictions)

    def save(self, path: str):
        if self.model is None:
            raise RuntimeError("You should train the model first")
        checkpoint = {
            "config": self.model.config,
            "trainer_config": self.config,
            "model_name": self.model.model_name,
            "model_state_dict": self.model.state_dict()
        }
        torch.save(checkpoint, path)

    @classmethod
    def load(cls, path: str):
        ckpt = torch.load(path)
        keys = ["config", "trainer_config", "model_state_dict"]
        for key in keys:
            if key not in ckpt:
                raise RuntimeError(f"Missing key {key} in checkpoint")
        new_model = ModelForClassification(
            ckpt['model_name'],
            ckpt["config"]
        )
        new_model.load_state_dict(ckpt["model_state_dict"])
        new_trainer = cls(ckpt["trainer_config"])
        new_trainer.model = new_model
        new_trainer.model.to(new_trainer.device)
        return new_trainer

In [64]:
class HabrDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_len, topic):
        self.data = dataframe
        self.text = dataframe['text'].tolist()
        self.targets = None
        if topic in dataframe:
            self.targets = dataframe[topic].tolist()
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_seq_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        if self.targets is not None:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
            }

    def __len__(self) -> int:
        return len(self.text)

In [65]:
MAX_LEN = 256
BATCH_SIZE = 128
MODEL = 'cointegrated/rubert-tiny2'
SEED = 42

In [66]:
torch.manual_seed(SEED)

In [78]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL, truncation=True, do_lower_case=True)

In [79]:
topics = ['admin', 'design', 'develop', 'management', 'marketing', 'popsci']

train_data, test_data = train_test_split(preprocessed_data_df, test_size=0.2,
                                         random_state=SEED)

In [80]:
train_data.shape, test_data.shape

((7563, 10), (1891, 10))

In [81]:
train_params = {"batch_size": BATCH_SIZE,
                "shuffle": True,
                "num_workers": 0
                }

test_params = {"batch_size": BATCH_SIZE,
               "shuffle": False,
               "num_workers": 0
               }

In [82]:
config = {
    "num_classes": 2,
    "dropout_rate": 0.1
}

In [86]:
trainer_config = {
    "lr": 3e-4,
    "n_epochs": 2,
    "weight_decay": 1e-6,
    "batch_size": BATCH_SIZE,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": SEED,
}

In [87]:
y_pred_list = {}

for topic in topics:
  train_dataset = HabrDataset(train_data, tokenizer, MAX_LEN, topic)
  test_dataset = HabrDataset(test_data, tokenizer, MAX_LEN, topic)

  train_dataloader = DataLoader(train_dataset, **train_params)
  test_dataloader = DataLoader(test_dataset, **test_params)

  model = ModelForClassification(MODEL, config=config)

  t = Trainer(trainer_config)

  t.fit(model, train_dataloader)

  y_pred_list[topic] = t.predict(test_dataloader)

Epoch 1/2


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 2/2


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1/2


...

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1/2


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 2/2


  0%|          | 0/60 [00:00<?, ?it/s]

In [88]:
y_pred_list

{'admin': array([0, 0, 0, ..., 0, 0, 0]),
 'design': array([1, 0, 0, ..., 0, 1, 0]),
 'develop': array([0, 0, 1, ..., 0, 1, 0]),
 'management': array([0, 0, 0, ..., 0, 0, 0]),
 'marketing': array([0, 0, 0, ..., 0, 0, 1]),
 'popsci': array([0, 0, 1, ..., 1, 0, 0])}

In [89]:
y_test = np.asarray(test_data[topics])
y_test

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0]])

In [90]:
y_pred = np.array([
  np.array([y_pred_list[topic][i] for topic in topics])
  for i in range(len(test_data))
])
y_pred

array([[0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [91]:
count_metrics(y_test, y_pred)

{'accuracy': 0.3707033315705976,
 'precision': 0.6682146542827657,
 'recall': 0.5007733952049497,
 'f1-micro': 0.5725022104332449}